In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
# sys.path.insert(0, "/home/bradhakrishnan/ECE276A_PR2/code")
sys.path.insert(0, "/home/bradhakrishnan/ECE276A_PR2/code/icp_warm_up/")

data_base_path= "/home/bradhakrishnan/ECE276A_PR2/code/icp_warm_up/"

In [3]:
import utils
# import icp_warm_up.test_icp as icp_test

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
import numpy as np
from utils import read_canonical_model, load_pc, visualize_icp_result

obj_name = 'drill' # drill or liq_container
num_pc = 4 # number of point clouds

source_pc = read_canonical_model(obj_name, base_path=data_base_path)

for i in range(num_pc):
    target_pc = load_pc(obj_name, i, base_path=data_base_path)
    
    # estimated_pose, you need to estimate the pose with ICP
    pose = np.eye(4)
    
    # # visualize the estimated result
    # visualize_icp_result(source_pc, target_pc, pose)
    break

In [17]:
from sklearn.neighbors import NearestNeighbors

def get_pose(init_R, init_P):
    init_T= np.zeros([4,4])
    init_T[:3,:3]= init_R
    init_T[:, -1]= np.hstack((init_P, np.array([1])))
    return init_T

def get_R_and_P(T):
    return T[:3,:3], T[:-1,-1]

def get_corres_points(target_pc_moved, source_pc):
    kn_obj=NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(target_pc_moved)
    dist, idx= kn_obj.kneighbors(source_pc)
    return dist.flatten().sum(), idx.flatten()

In [18]:
import transforms3d as t3d
from tqdm import tqdm
sampled_target_pc= target_pc[::10]
sampled_source_pc= source_pc[::10]
number_of_yaw_splits= 3
yaw_split= 2*np.pi/number_of_yaw_splits
max_iters= 1000


for ii in range(1, number_of_yaw_splits+1):
    yaw_val= yaw_split*ii
    print("YAW_VALUE :: ", yaw_val)
    init_R= t3d.euler.euler2mat(0,0, yaw_val)
    target_centroid= target_pc.mean(axis=0)
    source_centroid= source_pc.mean(axis=0)
    init_P= target_centroid- source_centroid
    init_T = get_pose(init_R, init_P)
    accumulated_T= init_T
    prev_dist= 0
    curr_R, curr_p= get_R_and_P(init_T)
    moved_pc= np.matmul(curr_R,sampled_target_pc.T).T+curr_p
    distace_res, matched_idxs= get_corres_points(moved_pc,sampled_source_pc)
    dist_between_corresponding_pc_pts= np.linalg.norm(sampled_source_pc-moved_pc[matched_idxs],axis=1)
    #filter points that are too far away
    filter_dist_threshold= 0.5
    filtered_idx = np.where(dist_between_corresponding_pc_pts < filter_dist_threshold)
    sampled_source_pc=sampled_source_pc[filtered_idx]
    moved_pc=moved_pc[matched_idxs][filtered_idx]
    for itr in tqdm(range(max_iters)):
        distace_res, matched_idxs= get_corres_points(moved_pc,sampled_source_pc)
        moved_centroid= moved_pc.mean(axis=0)
        sampled_source_centroid= sampled_source_pc.mean(axis=0)
        centered_sampled_source_pc = sampled_source_pc - sampled_source_centroid
        centered_sampled_moved_pc = moved_pc - moved_centroid
        Q = np.matmul(centered_sampled_source_pc.transpose(), centered_sampled_moved_pc[matched_idxs])
        U, S, Vt = np.linalg.svd(Q, full_matrices=True)
        curr_R= np.matmul(U, Vt)
        curr_p = sampled_source_centroid - np.matmul(curr_R, moved_centroid.T).T
    
        new_T= get_pose(curr_R, curr_p)
        accumulated_T= np.matmul(new_T, accumulated_T)
        moved_pc= np.matmul(curr_R,centered_sampled_moved_pc.T).T+curr_p
        
        if np.abs(distace_res- prev_dist) < 1e-6:
            print(accumulated_T , distace_res)
            break
        prev_dist= distace_res
    break

YAW_VALUE ::  2.0943951023931953


100%|██████████| 1000/1000 [00:02<00:00, 338.73it/s]


In [19]:
visualize_icp_result(source_pc, target_pc, pose)

[Open3D WARNING] GLFW Error: Failed to detect any supported platform
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


error: XDG_RUNTIME_DIR not set in the environment.


In [20]:
accumulated_T

array([[ 0.72455464, -0.56011031,  0.40161799,  1.15322851],
       [ 0.19471972,  0.72533391,  0.660284  , -0.38832314],
       [ 0.66113902,  0.4002089 , -0.63460857, -1.60961169],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

# ICP set up 
1. Take target and move as per our R and P ; i.e R*target_pc+P
2. Build correspondece array
3. Now do kabsch stuff, find Q ie; correlation
4. SVD => get new P and R
5. Somehow accumulate the pose estimates

(1169, 3)

In [50]:
np.matmul(curr_R, centered_sampled_moved_pc.T).T

array([[-2.47122421e-15, -1.70370445e-15, -3.67093210e-16],
       [-2.47122421e-15, -1.70370445e-15, -3.67093210e-16],
       [-2.47122421e-15, -1.70370445e-15, -3.67093210e-16],
       ...,
       [-2.47122421e-15, -1.70370445e-15, -3.67093210e-16],
       [-2.47122421e-15, -1.70370445e-15, -3.67093210e-16],
       [-2.47122421e-15, -1.70370445e-15, -3.67093210e-16]])

In [12]:
Q

array([[-1.34106354e-29,  2.99767144e-29,  4.33873498e-30],
       [-1.06496222e-29,  1.81438008e-29,  2.85962078e-30],
       [ 1.57772181e-30,  9.46633086e-30,  1.18329136e-30]])

In [29]:
target_pc.shape

(1169, 3)

In [10]:
import pickle
import sys
import time 
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as a3
import transforms3d
from transforms3d.euler import euler2mat, mat2euler, quat2euler
import open3d as o3d
from sklearn.neighbors import NearestNeighbors
from utils import read_canonical_model, load_pc, visualize_icp_result
import numpy.ma as ma

np.seterr(all="ignore")

def createPoseMatrix(R, p):
  p = np.atleast_2d(p).transpose()
  result = np.hstack((R, p))
  result = np.vstack((result, np.array([0,0,0,1])))
  return result

def trajectoryFromPose(pose):
    R = pose[:3, :3]
    x, y, z = mat2euler(R)
    p = pose[:2, 3]
    return np.hstack((p, z))

def transformVectorsByR(R, x):
    return np.transpose(np.matmul(R, x.transpose()))


def findPointCloudCentroid(pc):
    pointsCount = np.shape(pc)[0]
    sum = np.sum(pc, axis=0)
    return sum / pointsCount

def buildCorrespondenceArray(m, z, R, p):
    z = transformVectorsByR(R, z) + p
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(z)
    distances, indicies = nbrs.kneighbors(m) #indicies returns for each point in m, it gives the index of z that is the closest point
    distances = distances.flatten()
    indicies = indicies.flatten()
    return indicies, np.sum(distances) #return the correspondences and the summed error of all the pairs


def ICP(poseInit, source_pc, target_pc, maxIterations, downSample, visualize=False, distanceNearest = 0.2):

    R = poseInit[:3, :3]
    p = poseInit[:3, 3] #initialize R and p with guess pose

    source_pc_ = source_pc[0:-1:downSample] 
    target_pc_ = target_pc[0:-1:downSample] #downsample parameter takes subset of pointcloud to make icp faster

    target_pc_ = transformVectorsByR(R, target_pc_) + p #orient target to initial guess pose
    # import ipdb; ipdb.set_trace()
    distances = 0
    previousDistances = 0
    resultPose = poseInit

    correspondence, distances = buildCorrespondenceArray(source_pc_, target_pc_, np.identity(3), np.zeros(3)) #here we pass over identity pose because the point cloud has already been moved closer from the previous iteration's R and p
    # import ipdb; ipdb.set_trace()
    print("Dist:: ", distances)
    normsNew = np.linalg.norm(source_pc_ - target_pc_[correspondence], axis=1) #get distances between associated points
    indexPointsCloseEnough = np.where(normsNew < distanceNearest) #get index of point pairs that are close to each other

    source_pc_ = source_pc_[indexPointsCloseEnough] 
    target_pc_ = target_pc_[correspondence][indexPointsCloseEnough]
    #discard points that are far away from the target points since they probably don't actually have an association

    for i in range(maxIterations):
        
        correspondence, distances = buildCorrespondenceArray(source_pc_, target_pc_, np.identity(3), np.zeros(3)) #here we pass over identity pose because the point cloud has already been moved closer from the previous iteration's R and p
        
        if visualize == True:
            visualize_icp_result(source_pc_, target_pc_, np.identity(4), np.identity(4)) #for debug


        #print("For ICP iteration " + str(i) + ", the summed error between target and source is: " + str(distances))

        #execute Kabsch algorithm with found correspondences and no-correspondence points removed from both point clouds

        centerSource = findPointCloudCentroid(source_pc_)
        centerTarget = findPointCloudCentroid(target_pc_)

        centeredSourcePC = source_pc_ - centerSource
        centeredTargetPC = target_pc_ - centerTarget

        Q = np.matmul(centeredSourcePC.transpose(), centeredTargetPC[correspondence])

        U, S, Vh = np.linalg.svd(Q, full_matrices=True)
        R = np.matmul(U, Vh)
        p = centerSource - transformVectorsByR(R, centerTarget)

        iterationPose = createPoseMatrix(R, p)
        resultPose = np.matmul(iterationPose, resultPose) #multiply result pose with currently found iteration pose to accumulate poses
        target_pc_ = transformVectorsByR(R, target_pc_) + p #move target incrementally closer to source

        if np.abs(distances - previousDistances) < 0.0000000001: #if the change in error is not significant, stop algorithm
            finalDistance = distances #store the final summed norm error between the point clouds
            break
        
        previousDistances = distances
    print(resultPose, finalDistance)
    return resultPose, finalDistance #return both the estimated relative pose and converged summed error of the matched point clouds


def visualize_icp_result(source_pc, target_pc, source_pose, target_pose):
  '''
  Visualize the result of ICP
  source_pc: numpy array, (N, 3)
  target_pc: numpy array, (N, 3)
  pose: SE(4) numpy array, (4, 4)
  
  '''
  source_pcd = o3d.geometry.PointCloud()
  source_pcd.points = o3d.utility.Vector3dVector(source_pc.reshape(-1, 3))
  source_pcd.paint_uniform_color([0, 0, 1])

  source_pcd.transform(source_pose)

  target_pcd = o3d.geometry.PointCloud()
  #target_pcd.points = o3d.utility.Vector3dVector((np.matmul(target_pc,R) + p).reshape(-1, 3))
  target_pcd.points = o3d.utility.Vector3dVector(target_pc.reshape(-1, 3))
  target_pcd.paint_uniform_color([1, 0, 0])

  target_pcd.transform(target_pose)

  o3d.visualization.draw_geometries([source_pcd, target_pcd])

  
numberOfDiscreteYaws = int(4) #number of yaw amounts to test for best initial pose guess from 0 to 2*pi
incrementalYaw = (2 * np.pi) / numberOfDiscreteYaws

errors = np.empty(1)
poses = []

print("Matching point clouds with ICP using discretized yaws")

for yawGuess in range(numberOfDiscreteYaws):

    print("Current yaw guess in rads: " + str(yawGuess * incrementalYaw))

    RInit = transforms3d.euler.euler2mat(0,0,yawGuess * incrementalYaw) #create rotation matrix based on yaw guess
    centerSourceInit = findPointCloudCentroid(source_pc)
    centerTargetInit = findPointCloudCentroid(transformVectorsByR(RInit, target_pc)) #find the centroid AFTER rotation
    pInit = centerSourceInit - centerTargetInit #a good position estimate is the difference between the two centroids
    poseInit = createPoseMatrix(RInit, pInit)
    poseICP, summedError = ICP(poseInit, source_pc, target_pc, 1000, 10) #run ICP with max iterations of 1000 and downsample factor of 10
    errors = np.append(errors, summedError) #track errors for each yaw increment
    poses.append(poseICP) #track associated pose with error
    

# errors = errors[1:] #remove first element
# visualize_icp_result(source_pc, target_pc, np.identity(4), poses[np.argmin(errors)]) #show ICP pose with the minimal error


Matching point clouds with ICP using discretized yaws
Current yaw guess in rads: 0.0
Dist::  58.986466887691705
[[ 0.54593535  0.76781114  0.33529189 -0.34407122]
 [-0.65175371  0.64066889 -0.40590699  0.39594481]
 [-0.52647099  0.00307124  0.85018755  0.26745378]
 [ 0.          0.          0.          1.        ]] 30.823011686799294
Current yaw guess in rads: 1.5707963267948966
Dist::  56.26765557558488
[[-0.47431601 -0.86513063 -0.16301325  0.29603772]
 [ 0.69615097 -0.48193048  0.53208725 -0.42835456]
 [-0.53888603  0.13889567  0.83084887  0.2733437 ]
 [ 0.          0.          0.          1.        ]] 29.150709749461345
Current yaw guess in rads: 3.141592653589793
Dist::  57.194240123289525
[[-0.5631342  -0.74414436 -0.35934529  0.35531846]
 [ 0.61923064 -0.66794482  0.41279913 -0.3777121 ]
 [-0.54720497  0.00994369  0.83693957  0.28214317]
 [ 0.          0.          0.          1.        ]] 30.479983483035273
Current yaw guess in rads: 4.71238898038469
Dist::  52.41570260537423
[[